In [0]:
%sql

create schema if not exists rwd.bronze;
     

In [0]:
source_path = "/Volumes/rwd/bronze/myvolume/bronze/staging/cancer_patient/"
checkpoint_path = "/Volumes/rwd/bronze/myvolume/bronze/cancer_patient_raw/checkpoint/"
schema_location = "/Volumes/rwd/bronze/myvolume/bronze/cancer_patient_raw/schema/"


In [0]:
# Autoloader read
df = (spark.readStream
          .format("cloudFiles")
          .option("cloudFiles.format", "csv")
          .option("header", "true")
          .option("inferSchema", "true")
          .option("cloudFiles.maxFilesPerTrigger", 1)
          .option("cloudFiles.schemaLocation", schema_location)  
          .load(source_path)
     )

In [0]:
# Write Bronze table (append)
(df.drop("_rescued_data")
    .writeStream
    .format("delta")
    .option("checkpointLocation", checkpoint_path)
    .option("mergeSchema", "true")
    .outputMode("append")
    .trigger(availableNow=True)
    .toTable("rwd.bronze.cancer_patient_raw")
)

In [0]:
%sql

select * from rwd.bronze.cancer_patient_raw

Patient_ID,Name,Age,Gender,Cancer_Patient
P001,Asha Rao,52,Female,Yes
P002,Rajesh Kumar,61,Male,Yes
P003,Meena Iyer,47,Female,Yes
P004,Vikram Singh,68,Male,Yes
P005,Anita Sharma,55,Female,Yes
P006,Suresh Patel,59,Male,Yes
P007,Leela Nair,44,Female,Yes
P008,Arun Das,63,Male,Yes
P009,Kavita Joshi,50,Female,Yes
P010,Manoj Verma,57,Male,Yes


In [0]:
schema_df = spark.read.json("/Volumes/rwd/bronze/myvolume/bronze/cancer_patient_raw/schema/*")
display(schema_df)


_corrupt_record,dataSchemaJson,partitionSchemaJson
v1,null,null
null,"{""type"":""struct"",""fields"":[{""name"":""Patient_ID"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Age"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Gender"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Cancer_Patient"",""type"":""string"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[]}"


In [0]:
%sql
MERGE INTO rwd.bronze.cancer_patient AS target
USING rwd.bronze.cancer_patient_raw AS source
ON target.patient_id = source.patient_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,0,0,10


In [0]:
%sql
select * from rwd.bronze.cancer_patient

Patient_ID,Name,Age,Gender,Cancer_Patient
P001,Asha Rao,52,Female,Yes
P002,Rajesh Kumar,61,Male,Yes
P003,Meena Iyer,47,Female,Yes
P004,Vikram Singh,68,Male,Yes
P005,Anita Sharma,55,Female,Yes
P006,Suresh Patel,59,Male,Yes
P007,Leela Nair,44,Female,Yes
P008,Arun Das,63,Male,Yes
P009,Kavita Joshi,50,Female,Yes
P010,Manoj Verma,57,Male,Yes
